In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import Modules
import ast
import pandas as pd
from pandas_profiling import ProfileReport


import surprise
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import accuracy

from surprise.model_selection import train_test_split
from surprise.model_selection import LeaveOneOut
from surprise.model_selection import cross_validate
from surprise import KNNBaseline
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering

import matplotlib.pyplot as plt
import mplcyberpunk
%matplotlib inline

# plt.style.use('ggplot')
plt.style.use("cyberpunk")

import os
os.getcwd()

'/home/jovyan/work/Capstone_3/Skincare-Recommender'

In [3]:
# Import Custom Modules
from src.helpers import *

In [4]:
# Import Data
items = pd.read_csv('data/item_df.csv')
users = pd.read_csv('data/user_df.csv')

# Make Changes to ingredients to be evaluated as list
items['ingredients'] = items['ingredients'].apply(ast.literal_eval)
users.drop(users.columns[0], axis=1, inplace=True)
items.drop(items.columns[0], axis=1, inplace=True)

In [5]:
# Only parse out needed features
users_truncate = users[['user_id', 'item_name', 'star_rating']]

In [6]:
users_truncate.head()

,user_id,item_name,star_rating
0,harleen253,item_1,5
1,supremekaylap15,item_1,2
2,Alybug97,item_1,4
3,Mikachew,item_1,2
4,jkbean,item_1,2


In [8]:
# items['ingredients']
items.head()

,item_url,item_brand,item_name,n_loves,avg_stars,n_reviews,five_stars,four_stars,three_stars,two_stars,one_stars,price,review_corpus,ingredients,normal,dry,combination,oily,sensitive
0,https://www.sephora.com/product/protini-tm-pol...,brand_1,item_1,259473,4.0,4395,2557,540,402,405,491,68.0,"['Soooo at first when I started using this, it...","[1, 2-hexanediol, acetyl glutamine, alanine, a...",1,1,1,1,0
1,https://www.sephora.com/product/the-water-crea...,brand_2,item_2,233187,4.0,3030,1817,392,307,232,282,68.0,"[""The gold spatula changed my life - I can't b...","[alcohol, belamcanda chinensis root extract, b...",1,0,1,1,1
2,https://www.sephora.com/product/the-true-cream...,brand_3,item_3,191092,4.5,4527,3276,665,246,173,167,38.0,['Completely wrong for combo/oily skin. This c...,"[1, 2-hexanediol, acrylates, alchemilla vulgar...",1,1,1,1,0
3,https://www.sephora.com/product/cactus-water-m...,brand_4,item_4,26641,4.0,2221,1263,538,178,108,134,38.0,['The BEST moisturizer if you’re trying to imp...,"[1, 2-hexanediol, aloe barbadensis flower extr...",1,0,1,1,0
4,https://www.sephora.com/product/dramatically-d...,brand_5,item_5,80124,4.5,2619,1721,565,151,85,97,46.0,"[""It has a great orangey smell and it did hydr...",[sources of vitamin c:],1,1,1,1,1


# Feature Engineering